# 准备数据

In [1]:
import sys

sys.path.append('../')

from alignment_utils import one_file_per_process
from alignment_utils import extract_and_combine_numbers
from alignment_utils import longest_increasing_subsequence_index
from alignment_utils import find_answer_split_str
from alignment_utils import find_next_question_index
from alignment_utils import refine_answers
from alignment_utils import match_specific_from_end
from alignment_utils import answer_area_str_process
from alignment_utils import generate_answer_area_string

In [2]:
import os
import glob
from pathlib import Path

examination_paper_list = []

path = Path("./answer_markdown")

for file in path.glob("*.md"):
    with open(file, "r", encoding="utf-8") as f:
        examination_paper_list.append({
            "file_path": file,
            "text": one_file_per_process(f.read())
        })

# 创建寻找题目（题目以及对应的选项等）的方法

In [95]:
def get_all_question(text):
    # 拆分成行
    lines = text.splitlines()
    
    # 定义不准确的题目列表    
    inaccuracy_question = []
    
    # 从0的位置寻找第一道题
    index = find_next_question_index(0, lines)
    
    while index < len(lines):
        # 寻找下一个题目的index
        next_index = find_next_question_index(index, lines)
        
        inaccuracy_question.append("\n".join(lines[index: next_index]))
        index = next_index
    
    print( [extract_and_combine_numbers(topic) for topic in inaccuracy_question])
    # 通过"最长递增子序列"寻找每个精准的题目所在inaccuracy_question对应的下标
    all_question_indexs = longest_increasing_subsequence_index(inaccuracy_question)
    
    # 定义准确的题目列表    
    all_question = []
    # index为all_question_indexs的下标，all_question_indexs[index]为inaccuracy_question的下标
    for index, question_index in enumerate(all_question_indexs):
        if index == len(all_question_indexs) -1 :
            all_question.append("\n".join(inaccuracy_question[question_index:]))
            break
            
        all_question.append("\n".join(inaccuracy_question[question_index:all_question_indexs[index+1]]))
        
    # 尝试寻找用于分割答题区与答案区的字符串，返回值为int/str，如果是str则是分割的字符串
    # 本质是在"all_question[-1]"寻找答案关键字等字样
    answer_split_str = find_answer_split_str(all_question)
    
    if isinstance(answer_split_str, str):
        # 如果找到这个拆分的字符串了，则先把最后一道题的内容进行拆分
        all_question[-1] = all_question[-1].split(answer_split_str)[0]
      
    # 看看试卷的title是否出现在"all_question[-1]"位置，如果出现则删除
    all_question[-1] = all_question[-1].replace("\n"+text.splitlines()[0]+"\n","")
        
    return all_question, answer_split_str

# 创建测试样本

In [120]:
test_text = examination_paper_list[42]["text"]

In [121]:
test_text.splitlines()

['2004年普通高等学校招生全国统一考试',
 '数学（理工类）（福建卷）',
 '第Ⅰ卷（选择题 共60分）',
 '一、选择题：本大题共12小题，每小题5分，共60分.在每小题给出的四个选项中，只有一项是符合题目要求的.',
 '1．复数![](./notebook/image/media/image1.wmf)的值是 （ ）',
 'A．－1 B．1 C．－32 D．32',
 '2．tan15°+cot15°的值是 （ ）',
 'A．2 B．2+![](./notebook/image/media/image2.wmf) C．4 D．![](./notebook/image/media/image3.wmf)',
 '3．命题p：若a、b∈R，则\\|a\\|+\\|b\\|\\1是\\|a+b\\|\\1的充分而不必要条件；',
 '命题q：函数y=![](./notebook/image/media/image4.wmf)的定义域是（－∞，－1![](./notebook/image/media/image5.wmf)∪\\[3，+∞![](./notebook/image/media/image6.wmf).则 （ ）',
 'A．"p或q"为假 B．"p且q"为真',
 'C．p真q假 D．p假q真',
 '4．已知F~1~、F~2~是椭圆的两个焦点，过F~1~且与椭圆长轴垂直的直线交椭圆于A、B两点，若△ABF~2~是真正三角形，则这个椭圆的离心率是 （ ）',
 'A．![](./notebook/image/media/image7.wmf) B．![](./notebook/image/media/image8.wmf) C．![](./notebook/image/media/image9.wmf) D．![](./notebook/image/media/image10.wmf)',
 '5．已知m、n是不重合的直线，α、β是不重合的平面，有下列命题：',
 '①若m![](./notebook/image/media/image11.wmf)α，n∥α，则m∥n；',
 '②若m∥α，m∥β，则α∥β；',
 '③若α∩β=n，m∥n，则m∥α且m∥β；',
 '④若m⊥α，m⊥β，则α∥β.',
 '其中真命题的个数是 （ ）',

In [128]:
# split_str用于进行下一步的分析（有可能这份试卷是有答题区和答案区，也有可能每道题的答案都在题的下方，还有可能根本没有答案）
all_question, split_str = get_all_question(test_text)
print(f"split_str:\n{split_str}")
if split_str == -1:
    print("此试卷无答案")
all_question

[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 17, 18, 20, 21, 22]
split_str:
数学答案（理工类）（福建卷）


['1．复数![](./notebook/image/media/image1.wmf)的值是 （ ）\nA．－1 B．1 C．－32 D．32',
 '2．tan15°+cot15°的值是 （ ）\nA．2 B．2+![](./notebook/image/media/image2.wmf) C．4 D．![](./notebook/image/media/image3.wmf)',
 '3．命题p：若a、b∈R，则\\|a\\|+\\|b\\|\\1是\\|a+b\\|\\1的充分而不必要条件；\n命题q：函数y=![](./notebook/image/media/image4.wmf)的定义域是（－∞，－1![](./notebook/image/media/image5.wmf)∪\\[3，+∞![](./notebook/image/media/image6.wmf).则 （ ）\nA．"p或q"为假 B．"p且q"为真\nC．p真q假 D．p假q真',
 '4．已知F~1~、F~2~是椭圆的两个焦点，过F~1~且与椭圆长轴垂直的直线交椭圆于A、B两点，若△ABF~2~是真正三角形，则这个椭圆的离心率是 （ ）\nA．![](./notebook/image/media/image7.wmf) B．![](./notebook/image/media/image8.wmf) C．![](./notebook/image/media/image9.wmf) D．![](./notebook/image/media/image10.wmf)',
 '5．已知m、n是不重合的直线，α、β是不重合的平面，有下列命题：\n①若m![](./notebook/image/media/image11.wmf)α，n∥α，则m∥n；\n②若m∥α，m∥β，则α∥β；\n③若α∩β=n，m∥n，则m∥α且m∥β；\n④若m⊥α，m⊥β，则α∥β.\n其中真命题的个数是 （ ）\nA．0 B．1 C．2 D．3',
 '6．某校高二年级共有六个班级，现从外地转入4名学生，要安排到该年级的两个班级且每班安排2名，则不同的安排方案种数为 （ ）\nA．![](./notebook/image/media/image12.wmf) B．![](./notebook/image/m

# 准备对齐答案的方法（如果试卷存在答题区与答案区）

In [129]:
def get_all_answer_sequence(answer_area_string):
    lines = answer_area_string.splitlines()
    
    inaccuracy_answers = []

    index = find_next_question_index(0, lines)
    while index < len(lines):
        print(lines[index])
        print()
        next_index = find_next_question_index(index, lines)
       
        inaccuracy_answers.append("\n".join(lines[index: next_index]))
        index = next_index
    print("============")
#     for i in inaccuracy_answers:
#         print(i)
#         print()
        
#     print([extract_and_combine_numbers(topic) for topic in inaccuracy_answers])
    inaccuracy_answer_indexes = longest_increasing_subsequence_index(inaccuracy_answers)
   
    processed_inaccuracy_answers = []
    for index, answer_index in enumerate(inaccuracy_answer_indexes):
        if index == len(inaccuracy_answer_indexes) -1 :
            processed_inaccuracy_answers.append(inaccuracy_answers[answer_index])
            break
        processed_inaccuracy_answers.append("\n".join(inaccuracy_answers[answer_index:inaccuracy_answer_indexes[index+1]]))
        
    return refine_answers(processed_inaccuracy_answers)[::-1]

In [130]:
answer_str = generate_answer_area_string(test_text, split_str)

In [131]:
answer_str.splitlines()

['',
 '一、1.A 2.C 3.D 4.A 5.B 6.C 7.B 8.B 9.A 10.D 11.D 12.B',
 '二、13．4![](./notebook/image/media/image51.wmf) 14.1/2 15.1，3 16.2/3',
 '三、',
 '17\\. 本小题主要考查平面向量的概念和计算，三角函数的恒等变换及其图象变换的基本技能，考查运算能力.满分12分.',
 '解：（Ⅰ）依题设，f(x)=2cos^2^x+![](./notebook/image/media/image52.wmf)sin2x=1+2sin(2x+![](./notebook/image/media/image53.wmf)).',
 '由1+2sin(2x+![](./notebook/image/media/image54.wmf))=1－![](./notebook/image/media/image55.wmf)，得sin(2 x +![](./notebook/image/media/image56.wmf))=－![](./notebook/image/media/image57.wmf).',
 '∵-![](./notebook/image/media/image58.wmf)≤x≤![](./notebook/image/media/image59.wmf)，∴-![](./notebook/image/media/image60.wmf)≤2x+![](./notebook/image/media/image54.wmf)≤![](./notebook/image/media/image61.wmf)，∴2x+![](./notebook/image/media/image54.wmf)=-![](./notebook/image/media/image59.wmf)，',
 '即x=-![](./notebook/image/media/image62.wmf).',
 '（Ⅱ）函数y=2sin2x的图象按向量c=(m，n)平移后得到函数y=2sin2(x－m)+n的图象，即函数y=f(x)的图象.',
 '由（Ⅰ）得 f(x)=2sin2(x+![](./notebook/image/media/image63.wmf))+1.'

In [132]:
for i in get_all_answer_sequence(answer_area_str_process(answer_str)):
    print(i)
    print()

1.A 2.C 3.D 4.A 5.B 6.C 7.B 8.B 9.A 10.D 11.D 12.B

13．4![](./notebook/image/media/image51.wmf) 14.1/2 15.1，3 16.2/3

17\. 本小题主要考查平面向量的概念和计算，三角函数的恒等变换及其图象变换的基本技能，考查运算能力.满分12分.

18.本小题主要考查概率统计的基础知识，运用数学知识解决问题的能力.满分12分.

20.本小题主要考查建立函数关系式、数列求和、不等式的等基础知识，考查运用数学知识解决实际问题的能力.满分12分.

21.本小题主要考查函数的单调性，导数的应用和不等式等有关知识，考查数形结合及分类讨论思想和灵活运用数学知识分析问题和解决问题的能力.满分14分.

22\. 本题主要考查直线、抛物线、不等式等基础知识，求轨迹方程的方法，解析几何的基本思想和综合解题能力.满分12分.

1.A 

2.C 

3.D 

4.A 

5.B 

6.C 

7.B 

8.B 

9.A 

10.D 

11.D 

12.B

13．4![](./notebook/image/media/image51.wmf) 

14.1/2 

15.1，3 

16.2/3

17\. 本小题主要考查平面向量的概念和计算，三角函数的恒等变换及其图象变换的基本技能，考查运算能力.满分12分.
解：（Ⅰ）依题设，f(x)=2cos^2^x+![](./notebook/image/media/image52.wmf)sin2x=1+2sin(2x+![](./notebook/image/media/image53.wmf)).
由1+2sin(2x+![](./notebook/image/media/image54.wmf))=1－![](./notebook/image/media/image55.wmf)，得sin(2 x +![](./notebook/image/media/image56.wmf))=－![](./notebook/image/media/image57.wmf).
∵-![](./notebook/image/media/image58.wmf)≤x≤![](./notebook/image/media/imag

# 对齐

In [133]:
def alignment_answer(all_question, answer_str):
    questions_with_answer = []
    all_answer = get_all_answer_sequence(answer_area_str_process(answer_str))

    questions_map = {extract_and_combine_numbers(question):question for question in all_question} 
    answer_map = {extract_and_combine_numbers(answer):answer for answer in all_answer}
    
    for sequence_number in questions_map:
        questions_with_answer.append({"question":questions_map.get(sequence_number),"answer":answer_map.get(sequence_number, None)})
    return questions_with_answer

In [134]:
for row in alignment_answer(all_question, answer_str):
    print()
    print(row["question"])
    print("-------------------------------------------------------")
    print(f"{row['answer']}")
    print()
    print("============================================================")

1.A 2.C 3.D 4.A 5.B 6.C 7.B 8.B 9.A 10.D 11.D 12.B

13．4![](./notebook/image/media/image51.wmf) 14.1/2 15.1，3 16.2/3

17\. 本小题主要考查平面向量的概念和计算，三角函数的恒等变换及其图象变换的基本技能，考查运算能力.满分12分.

18.本小题主要考查概率统计的基础知识，运用数学知识解决问题的能力.满分12分.

20.本小题主要考查建立函数关系式、数列求和、不等式的等基础知识，考查运用数学知识解决实际问题的能力.满分12分.

21.本小题主要考查函数的单调性，导数的应用和不等式等有关知识，考查数形结合及分类讨论思想和灵活运用数学知识分析问题和解决问题的能力.满分14分.

22\. 本题主要考查直线、抛物线、不等式等基础知识，求轨迹方程的方法，解析几何的基本思想和综合解题能力.满分12分.


1．复数![](./notebook/image/media/image1.wmf)的值是 （ ）
A．－1 B．1 C．－32 D．32
-------------------------------------------------------
1.A 


2．tan15°+cot15°的值是 （ ）
A．2 B．2+![](./notebook/image/media/image2.wmf) C．4 D．![](./notebook/image/media/image3.wmf)
-------------------------------------------------------
2.C 


3．命题p：若a、b∈R，则\|a\|+\|b\|\1是\|a+b\|\1的充分而不必要条件；
命题q：函数y=![](./notebook/image/media/image4.wmf)的定义域是（－∞，－1![](./notebook/image/media/image5.wmf)∪\[3，+∞![](./notebook/image/media/image6.wmf).则 （ ）
A．"p或q"为假 B．"p且q"为真
C．p真q假 D．p假q真
---------------------------------------------